## Load dependencies

In [1]:
import os
import numpy as np
import pandas as pd
import photofitt
from photofitt import display
from photofitt.analysis import quantify_peaks, cumulative_activity

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Load UV mitosis data:
folder = "/Users/esti/Documents/PROYECTOS/PHX/DOCS/MANUSCRIPT/CODE/"
path_data = os.path.join(folder, "data/mitosis_CHO_UV_clean.csv")
data = pd.read_csv(path_data)
print(len(data))

# Change the name of exposure times
Con = [c for c in np.unique(data["Subcategory-02"]) if c.__contains__("UV")]

for i in Con:
    new_condition_name = i.split("UV")[1]
    data["Subcategory-02"][data["Subcategory-02"]==i] = new_condition_name
## Choose a path to store the data:
output_path = folder

95348


In [3]:
#new_data = pd.read_csv("/Users/esti/Documents/PROYECTOS/PHX/mitosis_mediated_data_itqb_3/CHO/results/scaled_1.5709_results/new-data/mitosis_counting.csv")
#new_data = new_data.rename(columns={'mitosis': 'Number of cells', 
#                            'mitosis_normalised': 'Norm. Number of cells'})
#index = new_data[new_data['frame'].isnull()].index.to_list()
#for i in index:
#    new_data['frame'].iloc[i]=0
#index_new_data = new_data.index.to_list()

#index_new = [i for i in index_new_data if i not in index]
#print(index_new)

#for i in index_new:
#    new_data['frame'].iloc[i]=4*(len(new_data['frame'].iloc[i]))
                                 
#new_datadata
#new_data.to_csv("/Users/esti/Documents/PROYECTOS/PHX/mitosis_mediated_data_itqb_3/CHO/results/scaled_1.5709_results/new-data/mitosis_counting.csv")

#Con = [c for c in np.unique(data["Subcategory-02"]) if c.__contains__("UV")]
#print(Con)

#for i in Con:
#    new_condition_name = i.split("UV")[1]
#    data["Subcategory-02"][data["Subcategory-02"]==i] = new_condition_name
    
#print(data)


## Estimation of cell densities for each field of view

Each well is 9.3 X 8.7 mm. Each FOV (the one in the final images) is 660 X 660 microns

- Area of each well: 80.91 mm^2
- Area of each FOV: 0.4356 mm ^2
- Total # of cells per well: 40000 per well
- Total# of cells per FOV: 215.35


In [4]:
light_power = 6.255662
well_area = 80.91
cell_density = 25000
fov_area = 660
fov_mm = (fov_area*0.001)**2
cell_mm = cell_density / well_area
cell_fov = cell_mm * fov_mm
print(cell_fov)

134.59399332591772


In [5]:
# Estimate the ligth dose
data = photofitt.utils.numerical_dose(data, column_name="Subcategory-02", power=light_power)

## Generate categorical variables for the light dose
light_dose = np.unique(data["Light dose"])
data["Light dose cat"] = ''
for l in light_dose:
    if l > 0:
        cat = np.str(np.round(l, decimals=1)) + " J/cm2"
    else:
        cat = 'non-synchro-0 J/cm2'
    
    data["Light dose cat"][data["Light dose"]==l] = cat
data["Light dose cat"][data["Subcategory-02"]=="Synchro"] = '0 J/cm2'
hue_order = ['non-synchro-0 J/cm2', '0 J/cm2', '0.2 J/cm2',
             '0.3 J/cm2', '0.6 J/cm2', '1.3 J/cm2', '2.5 J/cm2',
             '5.0 J/cm2', '6.3 J/cm2', '31.3 J/cm2', 
             '62.6 J/cm2', '93.8 J/cm2', '125.1 J/cm2', '156.4 J/cm2']

In [6]:
## Rename the columns to get the proper naming of the axis
data = data.rename(columns={'mitosis': 'Number of cells', 
                            'mitosis_normalised': 'Norm. Number of cells'}) 
## There is raw and smooth data. We show the smooth data (with a moving average of kernel 5) for the plots.
groups = np.unique(data["processing"])
## name of the variable to display in the plot.
y_var = "Number of cells" # "Norm. Number of cells"
cell_fov = 134.59
norm = True 
if norm is True:
    data[y_var + '_norm'] = data[y_var]/cell_fov
    plot_y_var = y_var + '_norm'
else:
    plot_y_var = y_var

fig_format = ["png", "pdf"]
for g in groups:
    # sns.set_style()
    ploting_data = data[data["processing"]==g].reset_index(drop=True)
    title = "UV illumination"
    condition = "Light dose cat" # "Subcategory-02"
    #hue_order = ['Control-sync', 'Synchro', '25ms', '50ms', '100ms', '200ms', '400ms', '800ms',
                 #'01sec', '05sec', '10sec', '15sec', '20sec', '25sec']
    for f in fig_format:
        format_extension = 'pdf'
        name = "{0}_{1}.{2}".format(plot_y_var, g, f)
        display.conditions(ploting_data, plot_y_var, title, condition, output_path, name,
                        hue_order=hue_order, palette=sns.color_palette("CMRmap_r", 17))
        


ValueError: Could not interpret value `None` for parameter `style`

<Figure size 700x400 with 0 Axes>

In [ ]:
## Order the data for the plot
#hue_order = ['Control-sync', 'Synchro', '25ms', '50ms', '100ms', '200ms', '400ms', '800ms',
#             '01sec', '05sec', '10sec', '15sec', '20sec', '25sec']

hue_order = ['non-synchro-0 J/cm2', '0 J/cm2', '0.2 J/cm2',
             '0.3 J/cm2', '0.6 J/cm2', '1.3 J/cm2', '2.5 J/cm2',
             '5.0 J/cm2', '6.3 J/cm2', '31.3 J/cm2', 
             '62.6 J/cm2', '93.8 J/cm2', '125.1 J/cm2', '156.4 J/cm2']

fig_format = ["png", "pdf"]
for f in fig_format:
    name = "{0}.{1}".format(plot_y_var, f) # name we want to give to the plot
    new_name="cells_vertical_subplots_{}".format(name)

    display.vertical_distributions(data, plot_y_var, output_path, 
                                   new_name, 
                                   raw="Light dose cat",
                                   hue="Light dose cat", 
                                   hue_order=hue_order, 
                                   palette=sns.color_palette("CMRmap_r", 17))
    

## Plot peak data

### Calculate the mitosis peaks for each video

In [ ]:
y_var = "Number of cells" # "Norm. Number of cells"
cell_fov = cell_fov # 134.59
norm = True 
if norm is True:
    data[y_var + '_norm'] = data[y_var]/cell_fov
    plot_y_var = y_var + '_norm'
else:
    plot_y_var = y_var
    
# Integrate the information about the temporal peaks of mitosis
data_peaks = quantify_peaks(data, plot_y_var)

data_peaks = photofitt.utils.numerical_dose(data_peaks, column_name="Subcategory-02", power=6.255662)
## Generate categorical variables for the light dose
light_dose = np.unique(data_peaks["Light dose"])
data_peaks["Light dose cat"] = ''
for l in light_dose:
    if l > 0:
        cat = np.str(np.round(l, decimals=1)) + " J/cm2"
    else:
        cat = 'non-synchro-0 J/cm2'
    
    data_peaks["Light dose cat"][data_peaks["Light dose"]==l] = cat
data_peaks["Light dose cat"][data_peaks["Subcategory-02"]=="Synchro"] = '0 J/cm2'

### Display the peak information for each condition

In [ ]:
#### Show each distribution with both violins and points

## Rename the columns to get the proper naming in the plots
data_peaks = data_peaks.rename(columns={'peak_time': 'Peak time point (min)', 
                            'delay_synchro': 'Delay w.r.t. synchronised cells (min)',
                           'proportional_delay_synchro': 'Proportional delay w.r.t. synchronised cells'})

x_var = ["Peak time point (min)", "Delay w.r.t. synchronised cells (min)",
         "Proportional delay w.r.t. synchronised cells", "Resistant cell decrease",
         "Proportional resistant decrease"]
fig_format = ["png", "pdf"]
for x in x_var:
    for f in fig_format:
        name = "{0}.{1}".format(x, f) # name we want to give to the plot
        new_name="peaktime_violin_horizontal_exposure_time{}".format(name)
 #       display.violinplots_horizontal(data_peaks,"Subcategory-02", x, output_path, new_name, hue_order,
 #                              palette=sns.color_palette("CMRmap_r", 17))
        display.violinplots_horizontal(data_peaks,'Light dose cat', x, output_path, new_name, hue_order,
                               palette=sns.color_palette("CMRmap_r", 17))
        
    

In [ ]:
data_peaks_regression = data_peaks[data_peaks["Exposure time"]>=0]
#data_peaks_regression = data_peaks_regression[data_peaks_regression["Exposure time"]<=1]
x_var = ["Peak time point (min)", "Delay w.r.t. synchronised cells (min)",
         "Proportional delay w.r.t. synchronised cells", "Resistant cell decrease",
         "Proportional resistant decrease"]
fig_format = ["png", "pdf"]
for x in x_var:
    for f in fig_format:
        name = "{0}.{1}".format(x, f) # name we want to give to the plot
        new_name="peaktime_regression_{}".format(name)
        display.regressionfit(data_peaks_regression, x, "Exposure time", output_path, new_name, 
                              hue="Subcategory-02", hue_order=hue_order, spline_order=2,
                              height=4, aspect=1, palette=sns.color_palette("CMRmap_r", 17))
        
data_peaks_regression = data_peaks_regression[data_peaks_regression["Exposure time"]<=1]
for x in x_var:
    for f in fig_format:
        name = "{0}.{1}".format(x, f) # name we want to give to the plot
        new_name="peaktime_regression_limited_{}".format(name)
        display.regressionfit(data_peaks_regression, x, "Exposure time", output_path, new_name, 
                              hue="Subcategory-02", hue_order=hue_order, spline_order=2,
                              height=4, aspect=1, palette=sns.color_palette("CMRmap_r", 17))


## Display cell activity after mitosis

In [ ]:
## Path to the cell growth data:
# Load UV mitosis data:
path_growth_data = os.path.join(folder, "data/dynamics_intensity_WL UV - high density.csv")
activity_metrics = pd.read_csv(path_growth_data)
activity_metrics = photofitt.utils.numerical_dose(activity_metrics, column_name="Subcategory-02", power=light_power)

## Generate categorical variables for the light dose
light_dose = np.unique(activity_metrics["Light dose"])
activity_metrics["Light dose cat"] = ''
for l in light_dose:
    if l > 0:
        cat = np.str(np.round(l, decimals=1)) + " J/cm2"
    else:
        cat = 'non-synchro-0 J/cm2'
    
    activity_metrics["Light dose cat"][activity_metrics["Light dose"]==l] = cat
activity_metrics["Light dose cat"][activity_metrics["Subcategory-02"]=="Synchro"] = '0 J/cm2'

hue_order = ['non-synchro-0 J/cm2', '0 J/cm2', '0.2 J/cm2',
             '0.3 J/cm2', '0.6 J/cm2', '1.3 J/cm2', '2.5 J/cm2',
             '5.0 J/cm2', '6.3 J/cm2', '31.3 J/cm2', 
             '62.6 J/cm2', '93.8 J/cm2', '125.1 J/cm2', '156.4 J/cm2']
activity_metrics = activity_metrics.rename(columns={'time_variance': 'Cell growth'})

In [ ]:
y_var = "Cell growth"
hue="Light dose cat" # "Subcategory-02"
format_extension = 'png'
name = "cell_growth.{}".format(format_extension)
title = "Cell activity after mitosis"
display.conditions(activity_metrics, y_var, 
                   title, 
                   hue, 
                   output_path, name, 
                   hue_order=hue_order,
                   palette=sns.color_palette("CMRmap_r", 17),
                   style=None)

### Analysis of the cell growth after the peak in mitosis
We will average the dynamics and compute the cummulative sum after mitosis to estimate the dynamics of the daughter cells growing

In [ ]:
activity_dataframe, activity_metrics = cumulative_activity(activity_metrics, data_peaks, use_starting_point="event peak")

#### Display the cummulative results

In [ ]:
y_var = "Cummulative cell growth"
format_extension = 'pdf'
name = "cell_cummulative_growth_from0.{}".format(format_extension)

#display.plot_conditions(dynamics_metrics, y_var, "Cummulative cell growth after mitosis", "Subcategory-02", 
#                        output_path, name, style_condition="Subcategory-01", hue_order=hue_order)

fig = plt.figure(figsize=(4, 5), constrained_layout=True)
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.plotting_context("paper")
sns.set_theme(style="ticks", rc=custom_params)
sns.set(font_scale=0.85)
# Plot the results per category
sns.lineplot(x="frame", y=y_var, hue="Subcategory-02", data=activity_metrics,
             palette=sns.color_palette("CMRmap_r", 17), linewidth=1.5, alpha=0.75, hue_order=hue_order)
# plt.ylabel(y_label)
plt.xlabel("Time (min)")
plt.yscale("linear")
#plt.legend(loc='upper right')
format_extension = name.split(".")[-1]
format_extension = 'pdf'
name = "cell_cummulative_growth.{}".format(format_extension)
fig.savefig(os.path.join(output_path, name), format=format_extension, transparent=True)
format_extension = 'png'
name = "cell_cummulative_growth.{}".format(format_extension)
fig.savefig(os.path.join(output_path, name), format=format_extension, transparent=True)


#### Display the violin plots of the averaged cell growth vertically distributed

In [ ]:
x_var = ['Mean activity', "Activity slope"]

for x in x_var:
    f, ax = plt.subplots(figsize=(4, 5), constrained_layout=True)
    custom_params = {"axes.spines.right": False, "axes.spines.top": False}
    sns.plotting_context("paper")
    sns.set_theme(style="ticks", rc=custom_params)
    sns.set(font_scale=1)
    ax.yaxis.set_tick_params(pad=-6) 
    sns.violinplot(data=activity_dataframe, palette=sns.color_palette("CMRmap_r", 17), 
                   inner="points", order=hue_order,
                   hue_order=hue_order, bw=.5, orient="h", 
                   y="Subcategory-02", 
                   x=x, width=1, linewidth=0.5)

    # Tweak the visual presentation
    ax.xaxis.grid(True, color="gray")

    ax.set(ylabel="Exposure time to UV radiation")
    ax.set(xlabel=x)
    sns.despine(trim=True, left=True)

    format_extension = 'png'
    name = "{0}.{1}".format(x, format_extension) # name we want to give to the plot
    new_name="peaktime_violin_horizontal_{}".format(name)
    f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True, dpi=300)
    format_extension = 'pdf'
    name = "{0}.{1}".format(x, format_extension) # name we want to give to the plot
    new_name="peaktime_violin_horizontal_{}".format(name)
    f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True,dpi=300)

In [ ]:
activitymotion_dataframe=activity_dataframe.reset_index(drop=True)

In [ ]:
activity_dataframe["Exposure time"] = 0.
for i in range(len(activity_dataframe)):
    s = activity_dataframe["Subcategory-02"].iloc[i]
    if s.__contains__("Control"):
        n = -0.001
    elif s.__contains__("Synchro"):
        n = 0.
    else:
        n = float(''.join(filter(str.isdigit, s)))
        if s.__contains__("ms"):
            n = 0.001*n
    activity_dataframe.loc[[i],'Exposure time'] = n

In [ ]:
activity_dataframe_regression = activity_dataframe[activity_dataframe["Exposure time"]>=0]
activity_dataframe_regression = activity_dataframe_regression[activitymotion_dataframe_regression["Exposure time"]<=1]
x_var = ["Peak time point (min)", "Delay w.r.t. synchronised cells (min)",
         "Proportional delay w.r.t. synchronised cells"]

x = 'Mean activity'
f = sns.catplot(
    data = activity_dataframe_regression, x="Exposure time", y=x, 
    #hue="Subcategory-02", hue_order=hue_order, palette=sns.color_palette("CMRmap_r", 17),
    native_scale=True, height=4, aspect=2
)
sns.regplot(
    data = activity_dataframe_regression, x="Exposure time", y=x,
    scatter=False, truncate=True, order=2, color="green",#6
)
sns.regplot(
    data = activity_dataframe_regression, x="Exposure time", y=x,
    scatter=False, truncate=True, order=1, color="yellow"#2
)
sns.regplot(
    data = activitymotion_dataframe_regression, x="Exposure time", y=x,lowess=True, 
    scatter=False, truncate=True, order=1, color="orange"
)

## Set style
custom_params = {"axes.spines.right": False, "axes.spines.top": False}    
sns.plotting_context("paper")
sns.set(font_scale=1)
sns.set_theme(style="whitegrid", rc=custom_params)

format_extension = 'png'
name = "{0}.{1}".format(x, format_extension) # name we want to give to the plot
new_name="growth_regression_firstrange_{}".format(name)
f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True, dpi=300)
format_extension = 'pdf'
name = "{0}.{1}".format(x, format_extension) # name we want to give to the plot
new_name="growth_regression_firstrange_{}".format(name)
f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True,dpi=300)

# plt.xlim([-0.2, 1.2])
# format_extension = 'png'
# name = "{0}.{1}".format(x, format_extension) # name we want to give to the plot
# new_name="growth_regression_firstrange_lim_{}".format(name)
# f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True, dpi=300)
# format_extension = 'pdf'
# name = "{0}.{1}".format(x, format_extension) # name we want to give to the plot
# new_name="growth_regression_firstrange_lim_{}".format(name)
# f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True,dpi=300)

## Plot the results for different wavelengths

In [ ]:
wl = ["UV", "475", "568", "630"]
data = None
for w in wl:
    new_data = pd.read_csv(os.path.join(folder, "data/mitosis_CHO_{}_clean.csv".format(w)))
    ## Rename the columns to get the proper naming of the axis
    new_data = new_data.rename(columns={'mitosis': 'Number of cells', 
                                'mitosis_normalised': 'Norm. Number of cells'})
    data_peaks = quantify_peaks(new_data, 'Number of cells')
    if data is None:
        data = data_peaks
    else:
        data = pd.concat([data, data_peaks]).reset_index(drop=True)
# Integrate the information about the temporal peaks of mitosis


In [ ]:
np.unique(data["Subcategory-01"])

In [ ]:
#### Show each distribution with both violins and points
wl=[ 'WL UV - high density', 'WL 475 - high density', 'WL 568 - high density','WL 630 - high density']
hue_order = ['Control-sync', 'Synchro', 'UV50ms', 'UV100ms', 'UV200ms', 'UV400ms', 'UV800ms']
             #'UV01sec', 'UV05sec', 'UV10sec', 'UV15sec', 'UV20sec', 'UV25sec'] 

## Rename the columns to get the proper naming in the plots
data = data.rename(columns={'peak_time': 'Peak time point (min)', 
                            'delay_synchro': 'Delay w.r.t. synchronised cells (min)',
                           'proportional_delay_synchro': 'Proportional delay w.r.t. synchronised cells'})

x_var = ["Peak time point (min)", "Delay w.r.t. synchronised cells (min)",
         "Proportional delay w.r.t. synchronised cells", "Resistant cell decrease",
         "Proportional resistant decrease"]

# Create an array with the colors you want to use
colors = [ "#bf00ff", "#3366ff", "#bfff00", "#ff6633"]# Set your custom color palette
palette = sns.color_palette(colors)

for x in x_var:
    f, ax = plt.subplots(figsize=(12, 3), constrained_layout=True)
    custom_params = {"axes.spines.right": False, "axes.spines.top": False}
    sns.plotting_context("paper")
    sns.set_theme(style="ticks", rc=custom_params)
    sns.set(font_scale=1)
    ax.yaxis.set_tick_params(pad=-6) 
    sns.violinplot(data=data, palette=palette,  #palette=sns.color_palette("husl", 14), 
                   bw=.5, orient="v", order=hue_order, hue_order=wl,
                   x="Subcategory-02", hue="Subcategory-01", alpha=1,
                   y=x, width=0.7, linewidth=0.5)

    # Tweak the visual presentation
    ax.xaxis.grid(True)
    plt.legend(loc="upper right")
    ax.set(xlabel="Exposure time to UV radiation")
    ax.set(ylabel=x)
    sns.despine(trim=True, left=True)

    format_extension = 'png'
    name = "{0}_wl.{1}".format(x, format_extension) # name we want to give to the plot
    new_name="peaktime_violin_horizontal_{}".format(name)
    f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True, dpi=300)
    format_extension = 'pdf'
    name = "{0}_wl.{1}".format(x, format_extension) # name we want to give to the plot
    new_name="peaktime_violin_horizontal_{}".format(name)
    f.savefig(os.path.join(output_path, new_name), format=format_extension, transparent=True,dpi=300)
